<a href="https://colab.research.google.com/github/alifzl/Zhaav-MINER-Scripts/blob/main/exm02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
from typing import Union
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import MinMaxScaler
from pyspark.pandas import *
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.sql import functions
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 49 kB/s 
     |████████████████████████████████| 199 kB 53.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=0ed5dca11fe20fcef785588cc225ea8a657f18a33da7b7016ee3221af5961625
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


# create sparksesion in order to use spark sql:

In [2]:
def createSparkSession(appname:str):
        '''
        create spark session rdd.
            Parameters:
            ---------- 
                    appname (str): The appliction name  
                    

            Returns:
                    SparkSession

        '''
        
        import pyspark
        from pyspark.sql import SparkSession
        spark = SparkSession.builder.appName(appname).getOrCreate()
        return spark

In [3]:
spark=createSparkSession("exp02")

## Read CSV file into DataFrame with schema and header


In [8]:
def readcsvfile(header:str,inferSchema:str,appname:str,path:str,sep:str=None):
        '''
        create dataframe by reading csv file.
            Parameters:
            ---------- 
                    header(str):  auto detect header of the file  ,
                  
                    inferSchema(str):  auto detect type of columns,
                    
                    appname(str):  The appliction name,
                    
                    path(str):    file path
                    
                    sep(str): columns separator

            Returns
        --------------
               dataframe( :class:`DataFrame) : return dataframe out of csv file

        '''
        spark=createSparkSession(appname)
        dataframe=spark.read.csv(path,header=header,inferSchema=inferSchema,sep=sep) 
        return dataframe

In [51]:
df=readcsvfile(header=True,inferSchema=True,appname="exp01",path='/content/legal_text_classification.csv',sep=',')

In [10]:
def show(dataframe,number:int=None,truncate:bool=None):
        '''
        show the dataframe .
        
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe
                    
            number(int) : optional, number of rows to show.
            
            truncate : bool, optional
            If set to ``True``, truncate strings longer than 20 chars by default.
            If set to a number greater than one, truncates long strings to length ``truncate``
            and align cells right.
            
            Returns
        --------------
               """Prints the first ``n`` rows to the console.

    

        

                '''
        dataframe.show(number,truncate)

## common command

### show the dataframe

In [11]:
show(df,5,False)

+-------+------------+------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|case_id|case_outcome|case_title                                                                                |case_text                                                              

### type of dataframe column

In [12]:
def dataframetype(dataframe):
    """Returns all column names and their data types as a list
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe
                    
        Returns
        --------------
               Prints types of the rows """
    return dataframe.dtypes

In [13]:
dataframetype(df)

[('case_id', 'string'),
 ('case_outcome', 'string'),
 ('case_title', 'string'),
 ('case_text', 'string')]

### list of the columns

In [14]:
def coloumes(dataframe):
    """Returns all column names  as a list
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe
                    
        Returns
        --------------
               Prints all column names """
    return dataframe.columns

In [15]:
coloumes(df)

['case_id', 'case_outcome', 'case_title', 'case_text']

### select and show some columns

In [16]:
def select(dataframe,coloumes:Union[list, str]):
    """select specfic columns by name and return new dataframe
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe
            
            coloumes(str/list): list of string coloumes names         

        Returns
        --------------
              dataframe_new(:class:`DataFrame ): return new dataframe
              
              
        Example
        -------------- 
            1- common select 
                
                dataframe_new=dataframe.select("name")


            
            
            2- also can slicing the dataframe with start and end
            
                dataframe_new=dataframe.select(dataframe.columns[start=10:end=20])"""
    dataframe_new=dataframe.select(coloumes)
    return dataframe_new

In [17]:
select(df,["case_id","case_outcome"]).show()

+-------+------------+
|case_id|case_outcome|
+-------+------------+
|  Case1|       cited|
|  Case2|       cited|
|  Case3|       cited|
|  Case4|       cited|
|  Case5|       cited|
|  Case6|       cited|
|  Case7|       cited|
|  Case8|       cited|
|  Case9|     applied|
| Case10|    followed|
| Case11|       cited|
| Case12|       cited|
| Case13|    followed|
| Case14|    followed|
| Case15|    followed|
| Case16|    followed|
| Case17|    followed|
| Case19|       cited|
| Case21| referred to|
| Case22|       cited|
+-------+------------+
only showing top 20 rows



### list of all rows

In [18]:
def collect(dataframe)->list:
  """ return all of dataframe as list
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe
            
                   

        Returns
        --------------
              ListData(list): list of all dataframe """

  ListData=dataframe.collect()
  return ListData

In [19]:
collect(df)[0:5]

[Row(case_id='Case1', case_outcome='cited', case_title='Alpine Hardwood (Aust) Pty Ltd v Hardys Pty Ltd (No 2) [2002] FCA 224 ; (2002) 190 ALR 121', case_text='Ordinarily that discretion will be exercised so that costs follow the event and are awarded on a party and party basis. A departure from normal practice to award indemnity costs requires some special or unusual feature in the case: Alpine Hardwood (Aust) Pty Ltd v Hardys Pty Ltd (No 2) [2002] FCA 224 ; (2002) 190 ALR 121 at [11] (Weinberg J) citing Colgate Palmolive Co v Cussons Pty Ltd (1993) 46 FCR 225 at 233 (Sheppard J).'),
 Row(case_id='Case2', case_outcome='cited', case_title='Black v Lipovac [1998] FCA 699 ; (1998) 217 ALR 386', case_text='"The general principles governing the exercise of the discretion to award indemnity costs after rejection by an unsuccessful party of a so called Calderbank letter were set out in the judgment of the Full Court in Black v Lipovac [1998] FCA 699 ; (1998) 217 ALR 386. In summary those pri

## Common filtering

### starts with

In [20]:
def startswith(dataframe,columns:str,value:str):
    """ return all row of dataframe as a new dataframe that starts with the value  
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe

            columns(str) : columns of dataframe
            
                   

        Returns
        --------------
              dataframe_new(:class:`DataFrame): dataframe of the value"""
    dataframe_new=dataframe.where(dataframe[columns].startswith(value))
    return dataframe_new

In [21]:
startswith(df,"case_text","A").show()

+-------+-------------+--------------------+--------------------+
|case_id| case_outcome|          case_title|           case_text|
+-------+-------------+--------------------+--------------------+
| Case19|        cited|Motor Terms Co Pt...|Assuming that the...|
| Case56|        cited|Australian Securi...|ASIC does not see...|
| Case65|  referred to|Meriton Apartment...|As to the annulme...|
| Case66|  referred to|Re Gargan; Ex par...|As I understood h...|
|Case108|        cited|Chu Kheng Lim v M...|A narrower view o...|
|Case118|        cited|Navarrete v Minis...|Allsop J applied ...|
|Case122|      applied|Project Blue Sky ...|An effective noti...|
|Case127|        cited|Taylor, Re; Ex pa...|A natural person ...|
|Case130|        cited|Vassis, Re; Ex pa...|A natural person ...|
|Case159|   considered|Australian Compet...|ACCC then made th...|
|Case161|  referred to|Odhiambo v Minist...|ACCC then made th...|
|Case162|   considered|Sunstate Airlines...|ACCC then made th...|
|Case264| 

### endswith

In [22]:
def endswith(dataframe,columns:str,value:str):
    """ return all row of dataframe as a new dataframe that ends with the value 
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe

            columns(str) : columns of dataframe
            
                   

        Returns
        --------------
              dataframe_new(:class:`DataFrame): dataframe of the value"""
    dataframe_new=dataframe.where(dataframe[columns].endswith(value))
    return dataframe_new

In [24]:
endswith(df,"case_text","d").show()

+-------+-------------+--------------------+--------------------+
|case_id| case_outcome|          case_title|           case_text|
+-------+-------------+--------------------+--------------------+
|Case134|  referred to|Australian Securi...|"Mr Spencer submi...|
|Case146|        cited|Kimberly-Clark Au...|"In any case, the...|
|Case152|        cited|No-Fume Ltd v Fra...|"In any case, the...|
|Case190|      applied|Ebner v Official ...|"I am satisfied t...|
|Case191|        cited|Re Refugee Review...|"While judges of ...|
|Case248|     followed|New South Wales v...|"On 14 November 2...|
|Case253|    discussed|Re Dingjan; Ex pa...|"The High Court i...|
|Case254|    discussed|Re Pacific Coal P...|"The High Court i...|
|Case349|        cited|Birdseye v Austra...|"In TNT Skypak In...|
|Case357|   considered|TNT Skypak Intern...|"In TNT Skypak In...|
|Case381|distinguished|Carapark Holdings...|"The Commissioner...|
|Case388|    discussed|Commissioner of T...|"In Commissioner ...|
|Case442| 

### filter by condition

In [25]:
def filt(dataframe,condition:str):
    """ return all row of dataframe as dataframe that filter by condition 
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe

            columns(str) : columns of dataframe
            
                   

        Returns:
        --------------
              dataframe_new(:class:`DataFrame): dataframe of the value
              
        Example:
        --------------
        dataframe_new = filt (df,df["Ward"]=="value") 

              """
        
        
    dataframe_new=dataframe.filter(condition)
    return dataframe_new

In [26]:
filt(df,df["case_outcome"]=="cited").show()

+-------+------------+--------------------+--------------------+
|case_id|case_outcome|          case_title|           case_text|
+-------+------------+--------------------+--------------------+
|  Case1|       cited|Alpine Hardwood (...|Ordinarily that d...|
|  Case2|       cited|Black v Lipovac [...|"The general prin...|
|  Case3|       cited|Colgate Palmolive...|Ordinarily that d...|
|  Case4|       cited|Dais Studio Pty L...|"The general prin...|
|  Case5|       cited|Dr Martens Austra...|The preceding gen...|
|  Case6|       cited|GEC Marconi Syste...|I accept that the...|
|  Case7|       cited|John S Hayes &amp...|The preceding gen...|
|  Case8|       cited|Seven Network Lim...|"On the question ...|
| Case11|       cited|Castlemaine Toohe...|Hexal Australia P...|
| Case12|       cited|R v McFarlane; Ex...|"quia timet proce...|
| Case19|       cited|Motor Terms Co Pt...|Assuming that the...|
| Case22|       cited|Minister for Immi...|is plain that, wh...|
| Case23|       cited|Mor

## sort data

### count of data in column

In [27]:
def groupbydesc(dataframe,columnname:str):
  """ return  a sorted dataframe by  columnname
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           

            columnname(str):select the column
        Returns:
        --------------
            df_sorted (:class:`DataFrame) : new sorted dataframe 
            

            """
  from pyspark.sql.functions import col
  df_sorted = dataframe.groupby(columnname).count().orderBy(col("count").desc())
  return df_sorted

In [29]:
groupbydesc(df,"case_outcome").show()

+-------------+-----+
| case_outcome|count|
+-------------+-----+
|        cited| 6834|
|  referred to| 2383|
|      applied| 1379|
|     followed| 1309|
|   considered| 1007|
|    discussed|  522|
|distinguished|  339|
|     affirmed|   66|
|      related|   53|
|     approved|   43|
+-------------+-----+



### ascending or descending sort data 

In [31]:
def sortascending(dataframe,coloumes:str,ascending:bool):
    """ return all sorted  dataframe with  ascending or descending method
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe

            columns(str) : columns of dataframe
            
            ascending(bool): if ascending equal to true it's sorted by ascending if it's equal to False sorted descending
            
                   

        Returns:
        --------------
            dataframe_asc (:class:`DataFrame) : dataframe of sort value"""

    dataframe_asc = dataframe.orderBy(coloumes,ascending=ascending) 
    return dataframe_asc

In [32]:
sortascending(df,"case_outcome",False).show()

+---------+------------+--------------------+--------------------+
|  case_id|case_outcome|          case_title|           case_text|
+---------+------------+--------------------+--------------------+
| Case7738|     related|Investa Propertie...|On 18 July last, ...|
|Case13253|     related|( Orion Telecommu...|On 22 August 2007...|
| Case7866|     related|KGL Health Pty Li...|"The background a...|
| Case8779|     related|Australian Securi...|"The next problem...|
| Case8780|     related|Australian Securi...|"The next problem...|
| Case8822|     related|People Telecom Li...|On 11 April 2007 ...|
| Case9170|     related|Logic Group Tyres...|"This is an appli...|
|Case10248|     related|BMG Poseidon Corp...|grounds under thi...|
|Case10632|     related|Bitech Engineerin...|unjustified threa...|
|Case10646|     related|SFE Corporation L...|                null|
|Case10899|     related|Citrus Queensland...|In their written ...|
|Case10900|     related|Citrus Queensland...|The following iss

### filter columns using is in

In [33]:
def isin (dataframe,columns:str,value:str):
   """ return  a  dataframe  where the columns contain Specific  value
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe

            columns(str) : columns of dataframe
            
            value(str): value in columns
                   

        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : datafram contain Specific  value"""
   dataframe_new = dataframe[dataframe[columns].isin(value)]
   return dataframe_new

In [35]:
isin(df,"case_outcome","affirmed").show(20)

+--------+------------+--------------------+--------------------+
| case_id|case_outcome|          case_title|           case_text|
+--------+------------+--------------------+--------------------+
| Case189|    affirmed|SZJXC v the Minis...|"This is an appea...|
| Case364|    affirmed|SBRF v Minister f...|"This is set out ...|
| Case920|    affirmed|Nathan v Capital ...|"appellant, Mr Ka...|
|Case1086|    affirmed|BRGAE of 2006 &am...|                null|
|Case1185|    affirmed|SZLSW v Minister ...|This is an appeal...|
|Case1269|    affirmed|SZNAL v Minister ...|"appellant is a c...|
|Case1601|    affirmed|Hughes v Australi...|"13 January 2006 ...|
|Case1631|    affirmed|Re Taxpayer and F...|The appellant tax...|
|Case1982|    affirmed|SZKNB v Minister ...|On 20 May 2008, I...|
|Case2081|    affirmed|Internet Centre o...|"On 5 November 20...|
|Case2929|    affirmed|Byrne v Repatriat...|A coronial inquir...|
|Case2953|    affirmed|SZKDY v Minister ...|"This is an appea...|
|Case3251|

## handeling null value


### list of null value count

In [36]:
def null_value_calc(dataframe)->list:
  """ return  a  list of null value
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           

        Returns:
        --------------
            (null_columns_counts) (list) : list of null value
            

            """
  from pyspark.sql.functions import col
  null_columns_counts=[]
  numrows=dataframe.count()
  for k in dataframe.columns:
    nullrows=dataframe.where(col(k).isNull()).count()
    if (nullrows>0):
      temp = k,nullrows,(nullrows/numrows)*100
      null_columns_counts.append(temp)
  return (null_columns_counts)

In [37]:
null_value_calc(df)

[('case_text', 101, 0.7247936849659132)]

###drop null values
How and Thresh and Subset is optional

In [52]:
def dropnnull(dataframe,thresh:int=None,how:str="any",subset:list=None):
  """ drop all missing data column in dataframe
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
            thresh(int,optional) : optional value

            how(str,optional) : include (any, all)
            
            subset(list,optional): list of the column
            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with drop column 
            
  """
  dataframe_new=dataframe.na.drop(how,thresh=thresh,subset=subset)
  return dataframe_new

In [53]:
df=dropnnull(df)

### Drop duplicate rows in a dataframe (distinct) 
columnsname is optional

In [54]:
def dropDuplicates(dataframe,columnsname:list=None):
  """ drop all duplicate  column in dataframe 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           

            
            column(list,otional):  column name 
            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with drop the duplicate data 
            
  """
  dataframe_new = dataframe.drop_duplicates(subset=columnsname)
  return dataframe_new

In [55]:
df=dropDuplicates(df)

## String Operations

###  words and space between

In [44]:
def wordsonly(dataframe,columnname:str):
    """ return dataframe with only words and space between.
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            columnname(str):  column name 

  

            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with trim a column
            
    """ 

    dataframe_new = dataframe.withColumn(columnname,regexp_replace(col(columnname), '[^A-Za-z]+', ' '))
    return dataframe_new

In [56]:
for item in df.columns[1:]:
  df=wordsonly(df,item)

In [62]:
df.show(5,truncate=False)

+-------+------------+-----------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
def wordsonly(dataframe,columnname:str):
    """ return dataframe with only words and space between.
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            columnname(str):  column name 

  

            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with trim a column
            
    """ 

    dataframe_new = dataframe.withColumn(columnname,regexp_replace(col(columnname), '[^A-Za-z]+', ' '))
    return dataframe_new

### substring the dataframe

In [58]:
def substrcolumn(dataframe,column:str,newcolumn:str,start:int,lenght:int):
    """ sub string a column and make new column
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            column(str):  column name 

            newcolumn(str): new column name 

            start (int): starting ponit
            
            lenght(int) : lenght of column 
            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with substring a column
            
    """
    dataframe_new = dataframe.withColumn(newcolumn, dataframe[column].substr(start, lenght))
    return dataframe_new

In [59]:
substrcolumn(df,"case_title","case_title2",0,2).show()

+--------+-------------+--------------------+--------------------+-----------+
| case_id| case_outcome|          case_title|           case_text|case_title2|
+--------+-------------+--------------------+--------------------+-----------+
|  Case41|        cited|Australian Securi...|Numerous cases ha...|         Au|
| Case351|        cited|Collector of Cust...| In Collector of ...|         Co|
| Case367|        cited|SZEPZ v Minister ...| i Until the Trib...|         SZ|
| Case390|        cited|Countess of Becti...| The applicants s...|         Co|
| Case440|  referred to|The Queen v Ross ...|On May Mrs Edge i...|         Th|
| Case546|  referred to|Paras v Public Se...|On May I dismisse...|         Pa|
| Case934|        cited|Re Minister for I...| Minister for Imm...|         Re|
|Case1365|        cited|GP International ...| In my view the s...|         GP|
|Case1488|  referred to|Latoudis v Casey ...| The power of the...|         La|
|Case1770|     followed|Strickland v Nati...| June a

### Trim the spaces from both ends for the specified string column.

In [61]:
def trimstrcolumn(dataframe,column,newcolumn:str):
    """ Trim the spaces from both ends for the specified string column.
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            column(str):  column name 

            newcolumn(str): new column name 


            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with trim a column
            
    """

    from pyspark.sql import functions as F
    dataframe = dataframe.withColumn(newcolumn, F.trim(column))
    return dataframe

In [65]:
df=trimstrcolumn(df,"case_text","case_text")

### left trim and right trim

In [66]:
def ltrim(dataframe,newcolumn:str,column:str):
    """ left Trim the spaces from string column.
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            column(str):  column name 

            newcolumn(str): new column name 


            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with trim a column
            
    """
    from pyspark.sql import functions as F
    dataframe_new = dataframe.withColumn(newcolumn, F.ltrim(dataframe[column]))
    return dataframe_new

In [67]:
def rtrim(dataframe,newcolumn:str,column:str):
    """ right Trim the spaces from string column.
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            column(str):  column name 

            newcolumn(str): new column name 


            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with trim a column
    """        
    from pyspark.sql import functions as F
    dataframe_new = dataframe.withColumn(newcolumn, F.rtrim(dataframe[column]))
    return dataframe_new

### Left-pad the string column to width len with pad

In [68]:
def leftpad(dataframe,column,newcolumn:str,character:str,digitnumber:int):
    """ Left-pad the string column to width len with pad.
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            column(str):  column name 

            newcolumn(str): new column name 

            character(str): a character to add a column

            digitnumber(int): number of character

            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with trim a column
            
    """

    from pyspark.sql import functions as F
    dataframe_new = dataframe.withColumn(newcolumn, F.lpad(dataframe[column], digitnumber, character))
    return dataframe_new